In [1]:
import pyodbc #подключение к SQL Server
import openpyxl #Работа с файлами excel
import pathlib #Работа с файлами и путями к ним
 #import user_agents #Работа с данными пользователя, например, выделение браузера
import pandas as pd #Работа с табличными данными
import matplotlib.pyplot as plt #Работа с графиками
import datetime #Работа с датами
import seaborn as sbnm #Работа с графиками
import numpy as np 
from scipy import stats as st
import openpyxl as ox
from openpyxl import Workbook
from openpyxl.worksheet import worksheet
import os
from pandas.io.excel import ExcelWriter
import json
import warnings  #Выключение предупреждений

In [2]:
# Settings the warnings to be ignored 
warnings.filterwarnings('ignore')

Driver='ODBC Driver 17 for SQL Server'
Server=r'srv-dwh-lsn,15025'
Database=['dwh','Reps','presentation']
# 'dwh' = Database[0]
# 'Reps' = Database[1]
# 'presentation' = Database[2]

def get_adw_connection():
    # Установка соединения 
    connection = pyodbc.connect('Driver=' + Driver + ';\
                                 Server=' + Server + ';\
                                 Database=' + Database[0] + ';\
                                 Trusted_Connection=yes')
    return connection

**ЗАГРУЖАЕМ ИЗ БД**

In [3]:
query = """
SET NOCOUNT ON; 

--======================================================== Квоты 

drop table if exists #dates_q		
	;with d as
		(select *, d.Year*100+Month 'MonthId', d.YearForWeek*100+WeekOfYear 'WeekId', d.YearForWeek*10000+WeekOfYear*100+DayOfWeek 'DayOfWeekId' 
		from dwh.dbo.dates d)	
	select * 
	into #dates_q
	from d	where 1=1 and --DateId between cast((convert(nvarchar, getdate(), 112) ) as int) and cast( (convert(varchar, getdate()+180, 112)) as int) 
	 DateId = cast((convert(nvarchar, getdate(), 112) ) as int)

drop table if exists #qotes	
select 
	 DateId 'Today'
	,p.Decription as 'Plng'
	,p.[PlanogrammExtId] as 'NumPlng' 
	,SiteFormat 'SiteFormat'
	,cast(PlngSize as nvarchar(max)) as 'Size'	
	,QTA_RG	
into #qotes	
from 
	(select PlngExtId, SiteFormat, PlngSize, AttributeClass, AttributeValue ,d.DateId, d.WeekOfYear
	from dwh.gld.ItemAttributes_Quota_data	
	join #dates_q d on d.DateId between StartDate and EndDate) a
pivot(sum(AttributeValue) 
	for AttributeClass in([QTA_RG],[QTA_IN]) ) pvt
	left join dwh.staging.Planogramms p on p.PlanogrammExtId = pvt.PlngExtId	

drop table if exists #QTA1 
select q.Today,  q.Plng, q.SiteFormat,Size, q.NumPlng, QTA_RG 
into #QTA1
from #qotes q 
where QTA_RG is not NULL  and QTA_RG!=0

select * from #QTA1


"""

gp=get_adw_connection()
quots = pd.read_sql(query, con=gp)
gp.close()

In [4]:
quots[quots['Plng'] == 'МУЧНАЯ КОНДИТЕРКА']

,Today,Plng,SiteFormat,Size,NumPlng,QTA_RG
188,20250321,МУЧНАЯ КОНДИТЕРКА,DIXY,1,125,43.0
189,20250321,МУЧНАЯ КОНДИТЕРКА,DIXY,2,125,102.0
190,20250321,МУЧНАЯ КОНДИТЕРКА,DIXY,3,125,155.0
191,20250321,МУЧНАЯ КОНДИТЕРКА,DIXY,4,125,180.0
192,20250321,МУЧНАЯ КОНДИТЕРКА,DIXY,5,125,205.0
193,20250321,МУЧНАЯ КОНДИТЕРКА,DIXY,20,125,35.0
194,20250321,МУЧНАЯ КОНДИТЕРКА,DIXY,21,125,49.0
195,20250321,МУЧНАЯ КОНДИТЕРКА,DIXY,50,125,102.0
196,20250321,МУЧНАЯ КОНДИТЕРКА,DIXY,77,125,205.0


In [43]:
%%time
#SET NOCOUNT ON; 

query = """

SELECT 

case when s.level3_descr = 'СЕВЕРО-ЗАПАДНЫЙ' then 'СЗФО' else 'ЦФО' end 'FO'
,case
	when level6_descr in ('M','S','L','XL') then 'GO' 
	when level6_descr='TOWER' then 'TOWER' 
	when level6_descr='DARKSTORE' then 'DARKSTORE' 
	else 'DIXY' end 'SiteFormat' 	
,PlngSize 'PLNG_SIZE'
,PlngDescription 'Plng'  
,count(distinct s.SiteExtId) 'Count_TT_max'

FROM [DWH].[GLD].[TypeSettingMatrixPeriod] mp
JOIN [DWH].[GLD].[Attributes] a on mp.AttributeID = a.AttributeId
JOIN [DWH].[dbo].[DProducts] d on mp.ItemId = d.ItemId
JOIN [DWH].[dbo].[DNodes] s on mp.SiteId = s.SiteID

WHERE 1=1 and format(getdate(), 'yyyyMMdd') between mp.DateFrom and mp.DateTo
--and PlngSize='H2' and PlngDescription = 'ВИНО'
group by PlngDescription, PlngSize, case when s.level3_descr = 'СЕВЕРО-ЗАПАДНЫЙ' then 'СЗФО' else 'ЦФО' end
,case
	when level6_descr in ('M','S','L','XL') then 'GO' 
	when level6_descr='TOWER' then 'TOWER' 
	when level6_descr='DARKSTORE' then 'DARKSTORE' 
	else 'DIXY' end
Order by 2,3,4

"""
gp=get_adw_connection()
all_plng = pd.read_sql(query, con=gp)
gp.close()



query = """

SELECT 
case when s.level3_descr = 'СЕВЕРО-ЗАПАДНЫЙ' then 'СЗФО' else 'ЦФО' end 'FO'
,case
	when level6_descr in ('M','S','L','XL') then 'GO' 
	when level6_descr='TOWER' then 'TOWER' 
	when level6_descr='DARKSTORE' then 'DARKSTORE' 
	else 'DIXY' end 'SiteFormat' ,	
D.Level2_desc 'Уровень 2', 
D.Level3_desc 'Уровень 3', 
D.Level4_desc 'Уровень 4', 
D.Level5_desc 'Уровень 5', 
D.g_ItemCode 'Item', 
PlngDescription 'Plng', 
PlngSize 'Plng_Size', 
count(DISTINCT s.SiteExtID) 'Count_TT'

FROM 
[DWH].[GLD].[TypeSettingMatrixPeriod] mp
JOIN [DWH].[GLD].[Attributes] a on mp.AttributeID = a.AttributeId
JOIN [DWH].[dbo].[DProducts] d on mp.ItemId = d.ItemId
JOIN [DWH].[dbo].[DNodes] s on mp.SiteId = s.SiteID	--JOIN Dates da on da.DateId=mp.DateFrom	
    
WHERE 
1=1 and 
format(getdate(), 'yyyyMMdd') between mp.DateFrom and mp.DateTo and 
InOut in ('Постоянный', 'Временный') and AssortTypeName in ('РЕГУЛЯРНЫЙ', 'СЕЗОННЫЙ','РАСПРОДАЖА')
and concat(InOut,' ',AssortTypeName) != 'Временный РЕГУЛЯРНЫЙ'
--and InOut not in ('Временный')

GROUP BY
case when s.level3_descr = 'СЕВЕРО-ЗАПАДНЫЙ' then 'СЗФО' else 'ЦФО' end, 
D.Level2_desc, D.Level3_desc, D.Level4_desc, D.Level5_desc, 
D.g_ItemCode, PlngDescription, PlngSize ,case
	when level6_descr in ('M','S','L','XL') then 'GO' 
	when level6_descr='TOWER' then 'TOWER' 
	when level6_descr='DARKSTORE' then 'DARKSTORE' 
	else 'DIXY' end  

"""

gp=get_adw_connection()
now = pd.read_sql(query, con=gp)
gp.close()

query = """

SELECT 
case when s.level3_descr = 'СЕВЕРО-ЗАПАДНЫЙ' then 'СЗФО' else 'ЦФО' end 'FO'
,case
	when level6_descr in ('M','S','L','XL') then 'GO' 
	when level6_descr='TOWER' then 'TOWER' 
	when level6_descr='DARKSTORE' then 'DARKSTORE' 
	else 'DIXY' end 'SiteFormat',
D.Level2_desc 'Уровень 2', 
D.Level3_desc 'Уровень 3', 
D.Level4_desc 'Уровень 4', 
D.Level5_desc 'Уровень 5', 
D.g_ItemCode 'Item', 
PlngDescription 'Plng', 
PlngSize 'Plng_Size', 
da.WeekOfYear 'Признак', 
count(DISTINCT s.SiteExtID) 'Count_TT'

FROM 
[DWH].[GLD].[TypeSettingMatrixPeriod] mp
JOIN [DWH].[GLD].[Attributes] a on mp.AttributeID = a.AttributeId
JOIN [DWH].[dbo].[DProducts] d on mp.ItemId = d.ItemId
JOIN [DWH].[dbo].[DNodes] s on mp.SiteId = s.SiteID
JOIN (Select DateId, WeekOfYear  
    from [DWH].[dbo].[Dates] da where DateId > format(getdate(), 'yyyyMMdd')) da on da.DateId=mp.DateFrom

WHERE 
1=1 and 
InOut in ('Постоянный', 'Временный') and 
AssortTypeName in ('РЕГУЛЯРНЫЙ', 'СЕЗОННЫЙ', 'РАСПРОДАЖА') 
and (LEFT(mp.DateFrom,4)= DATEPART(YYYY, getdate()) OR LEFT(mp.DateFrom,4)= DATEPART(YYYY, getdate())+1)
and concat(InOut,' ',AssortTypeName) != 'Временный РЕГУЛЯРНЫЙ'
--and InOut not in ('Временный')

GROUP BY 
case when s.level3_descr = 'СЕВЕРО-ЗАПАДНЫЙ' then 'СЗФО' else 'ЦФО' end,
D.Level2_desc, 
D.Level3_desc, 
D.Level4_desc, 
D.Level5_desc, 
D.g_ItemCode, 
PlngDescription, 
PlngSize , 
da.WeekOfYear
,case
	when level6_descr in ('M','S','L','XL') then 'GO' 
	when level6_descr='TOWER' then 'TOWER' 
	when level6_descr='DARKSTORE' then 'DARKSTORE' 
	else 'DIXY' end 

"""

gp=get_adw_connection()
vvod = pd.read_sql(query, con=gp)
gp.close()

query = """

SELECT 
case when s.level3_descr = 'СЕВЕРО-ЗАПАДНЫЙ' then 'СЗФО' else 'ЦФО' end 'FO'
,case
	when level6_descr in ('M','S','L','XL') then 'GO' 
	when level6_descr='TOWER' then 'TOWER' 
	when level6_descr='DARKSTORE' then 'DARKSTORE' 
	else 'DIXY' end 'SiteFormat',
D.Level2_desc 'Уровень 2', 
D.Level3_desc 'Уровень 3',
D.Level4_desc 'Уровень 4', 
D.Level5_desc 'Уровень 5', 
D.g_ItemCode 'Item', 
PlngDescription 'Plng', 
PlngSize 'Plng_Size', 
da.WeekOfYear 'Признак', 
count(DISTINCT s.SiteExtID) 'Count_TT'

FROM 
[DWH].[GLD].[TypeSettingMatrixPeriod]  mp
JOIN [DWH].[GLD].[Attributes] a on mp.AttributeID = a.AttributeId
JOIN [DWH].[dbo].[DProducts] d on mp.ItemId = d.ItemId
JOIN [DWH].[dbo].[DNodes] s on mp.SiteId = s.SiteID
JOIN [DWH].[dbo].[Dates] da on da.DateId=mp.DateTo

WHERE 
1=1 and 
InOut in ('Постоянный', 'Временный') and 
AssortTypeName in ('РЕГУЛЯРНЫЙ', 'СЕЗОННЫЙ', 'РАСПРОДАЖА')
and da.DateId>format(getdate(), 'yyyyMMdd')
--and LEFT(mp.DateTo,4)= DATEPART(YYYY, getdate())
and (LEFT(mp.DateTo,4)= DATEPART(YYYY, getdate()) OR LEFT(mp.DateTo,4)= DATEPART(YYYY, getdate())+1)
and concat(InOut,' ',AssortTypeName) != 'Временный РЕГУЛЯРНЫЙ'
--and InOut not in ('Временный')

GROUP BY
case when s.level3_descr = 'СЕВЕРО-ЗАПАДНЫЙ' then 'СЗФО' else 'ЦФО' end ,
D.Level2_desc, 
D.Level3_desc,
D.Level4_desc, 
D.Level5_desc, 
D.g_ItemCode, 
PlngDescription, 
PlngSize, 
da.WeekOfYear
,case
	when level6_descr in ('M','S','L','XL') then 'GO' 
	when level6_descr='TOWER' then 'TOWER' 
	when level6_descr='DARKSTORE' then 'DARKSTORE' 
	else 'DIXY' end 

"""

gp=get_adw_connection()
vivod = pd.read_sql(query, con=gp)
gp.close()


CPU times: total: 3.86 s
Wall time: 3min 46s


In [6]:
all_plng.sample()

,FO,SiteFormat,PLNG_SIZE,Plng,Count_TT_max
792,ЦФО,DIXY,L3,СЛАДОСТИ,137


In [7]:
now.sample()

,FO,SiteFormat,Уровень 2,Уровень 3,Уровень 4,Уровень 5,Item,Plng,Plng_Size,Count_TT
17723,СЗФО,DIXY,БАКАЛЕЯ,БАКАЛЕЙНЫЕ ТОВАРЫ,"СПЕЦИИ, ПРИПРАВЫ",ПРИПРАВЫ,2000351652,СПЕЦИИ,M0,7


In [8]:
vvod.sample()

,FO,SiteFormat,Уровень 2,Уровень 3,Уровень 4,Уровень 5,Item,Plng,Plng_Size,Признак,Count_TT
990,СЗФО,DIXY,НОНФУД 1,"ТОВАРЫ ДЛЯ ДЕТЕЙ, БЕРЕМЕННЫХ И КОРМЯЩИХ",ДЕТСКАЯ ГИГИЕНА,ПОДГУЗНИКИ - ТРУСИКИ,2000606932,ДЕТСКАЯ ГИГИЕНА,T2,13,1


In [9]:
vivod.sample()

,FO,SiteFormat,Уровень 2,Уровень 3,Уровень 4,Уровень 5,Item,Plng,Plng_Size,Признак,Count_TT
5341,ЦФО,DARKSTORE,НОНФУД 2,СЕЗОННЫЕ ТОВАРЫ,ПАСХА,ПИЩЕВЫЕ КРАСИТЕЛИ,2000214096,СПЕЦ ОБОР,D77,22,1


In [10]:
vivod.query('Plng=="ТАБАЧНЫЙ ШКАФ"')

,FO,SiteFormat,Уровень 2,Уровень 3,Уровень 4,Уровень 5,Item,Plng,Plng_Size,Признак,Count_TT
130,СЗФО,DIXY,"АЛКОГОЛЬ, НАПИТКИ, ТАБАК",ТАБАК,СИГАРЕТЫ,СИГАРЕТЫ,2000004109,ТАБАЧНЫЙ ШКАФ,M1,13,48
131,СЗФО,DIXY,"АЛКОГОЛЬ, НАПИТКИ, ТАБАК",ТАБАК,СИГАРЕТЫ,СИГАРЕТЫ,2000004109,ТАБАЧНЫЙ ШКАФ,M2,13,140
132,СЗФО,DIXY,"АЛКОГОЛЬ, НАПИТКИ, ТАБАК",ТАБАК,СИГАРЕТЫ,СИГАРЕТЫ,2000004109,ТАБАЧНЫЙ ШКАФ,M3,13,263
137,СЗФО,DIXY,"АЛКОГОЛЬ, НАПИТКИ, ТАБАК",ТАБАК,СИГАРЕТЫ,СИГАРЕТЫ,2000267139,ТАБАЧНЫЙ ШКАФ,M1,12,22
138,СЗФО,DIXY,"АЛКОГОЛЬ, НАПИТКИ, ТАБАК",ТАБАК,СИГАРЕТЫ,СИГАРЕТЫ,2000267139,ТАБАЧНЫЙ ШКАФ,M2,12,92
141,СЗФО,DIXY,"АЛКОГОЛЬ, НАПИТКИ, ТАБАК",ТАБАК,СИГАРЕТЫ,СИГАРЕТЫ,2000303185,ТАБАЧНЫЙ ШКАФ,M1,13,26
147,СЗФО,DIXY,"АЛКОГОЛЬ, НАПИТКИ, ТАБАК",ТАБАК,СИГАРЕТЫ,СИГАРЕТЫ,2000556791,ТАБАЧНЫЙ ШКАФ,M1,13,48
148,СЗФО,DIXY,"АЛКОГОЛЬ, НАПИТКИ, ТАБАК",ТАБАК,СИГАРЕТЫ,СИГАРЕТЫ,2000556791,ТАБАЧНЫЙ ШКАФ,M2,13,140
149,СЗФО,DIXY,"АЛКОГОЛЬ, НАПИТКИ, ТАБАК",ТАБАК,СИГАРЕТЫ,СИГАРЕТЫ,2000556791,ТАБАЧНЫЙ ШКАФ,M3,13,263
151,СЗФО,DIXY,"АЛКОГОЛЬ, НАПИТКИ, ТАБАК",ТАБАК,СИГАРЕТЫ,СИГАРЕТЫ,2000564871,ТАБАЧНЫЙ ШКАФ,M1,15,48


In [11]:
quots['Plng']=quots['Plng'].str.replace('ТАБАКОШОП', 'ТАБАЧНЫЙ ШКАФ')
quots.query('Plng=="ТАБАЧНЫЙ ШКАФ"')

,Today,Plng,SiteFormat,Size,NumPlng,QTA_RG
430,20250321,ТАБАЧНЫЙ ШКАФ,DIXY,1,160,120.0
431,20250321,ТАБАЧНЫЙ ШКАФ,DIXY,2,160,132.0
432,20250321,ТАБАЧНЫЙ ШКАФ,DIXY,3,160,144.0
433,20250321,ТАБАЧНЫЙ ШКАФ,DIXY,20,160,144.0


In [12]:
all_plng['PLNG_SIZE'] = all_plng['PLNG_SIZE'].replace("М3", "M3") #all_plng['PLNG_SIZE'].unique()
now['Plng_Size'] = now['Plng_Size'].replace("М3", "M3") #now['Plng_Size'].unique()
vvod['Plng_Size'] = vvod['Plng_Size'].replace("М3", "M3") #vvod['Plng_Size'].unique()
vivod['Plng_Size'] = vivod['Plng_Size'].replace("М3", "M3") #vivod['Plng_Size'].unique()

In [13]:
all_plng = all_plng.rename(columns = {'PLNG_SIZE':'Plng_Size'})
vivod_plng = pd.merge(vivod, all_plng, on=['FO', 'Plng', 'Plng_Size','SiteFormat'], how='inner')
vivod_plng['%TT'] = vivod_plng['Count_TT'] / vivod_plng['Count_TT_max']

vvod_plng = pd.merge(vvod, all_plng, on=['FO', 'Plng', 'Plng_Size','SiteFormat'], how='inner')
vvod_plng['%TT'] = vvod_plng['Count_TT'] / vvod_plng['Count_TT_max']


In [14]:
vivod.sample()

,FO,SiteFormat,Уровень 2,Уровень 3,Уровень 4,Уровень 5,Item,Plng,Plng_Size,Признак,Count_TT
4363,ЦФО,TOWER,БАКАЛЕЯ,ШОКОЛАД,ШОКОЛАД,ШОКОЛАД МОЛОЧНЫЙ,2000609791,ШОКОЛАД,G50,18,1


In [15]:
#now.head(3)

now_plng = pd.merge(now, all_plng, on=['FO', 'Plng', 'Plng_Size','SiteFormat'], how='inner')
now_plng['%TT'] = now_plng['Count_TT'] / now_plng['Count_TT_max']



In [16]:
now_plng.query('Plng == "ПРИКАССА"')

,FO,SiteFormat,Уровень 2,Уровень 3,Уровень 4,Уровень 5,Item,Plng,Plng_Size,Count_TT,Count_TT_max,%TT
17670,СЗФО,DIXY,"АЛКОГОЛЬ, НАПИТКИ, ТАБАК",ТАБАК,ТАБАЧНЫЕ АКСЕССУАРЫ,ЗАЖИГАЛКИ,2000535651,ПРИКАССА,H3,45,45,1.0
17671,СЗФО,DIXY,"АЛКОГОЛЬ, НАПИТКИ, ТАБАК",ТАБАК,ТАБАЧНЫЕ АКСЕССУАРЫ,ЗАЖИГАЛКИ,2000547911,ПРИКАССА,H3,45,45,1.0
17672,СЗФО,DIXY,"АЛКОГОЛЬ, НАПИТКИ, ТАБАК",ТАБАК,ТАБАЧНЫЕ АКСЕССУАРЫ,ЗАЖИГАЛКИ,2000553931,ПРИКАССА,H3,45,45,1.0
17673,СЗФО,DIXY,БАКАЛЕЯ,БАКАЛЕЙНЫЕ ТОВАРЫ,ДИЕТИЧЕСКОЕ ПИТАНИЕ,БАТОНЧИКИ ДИЕТИЧЕСКИЕ,2000418186,ПРИКАССА,H3,45,45,1.0
17674,СЗФО,DIXY,БАКАЛЕЯ,БАКАЛЕЙНЫЕ ТОВАРЫ,ДИЕТИЧЕСКОЕ ПИТАНИЕ,БАТОНЧИКИ ДИЕТИЧЕСКИЕ,2000608432,ПРИКАССА,H3,45,45,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
112299,ЦФО,TOWER,БАКАЛЕЯ,ПРИКАССА,ШОКОЛАД (П),ШОКОЛАДНЫЕ БАТОНЧИКИ (П),2000601791,ПРИКАССА,G50,1,1,1.0
112300,ЦФО,TOWER,БАКАЛЕЯ,ПРИКАССА,ШОКОЛАД (П),ШОКОЛАДНЫЕ БАТОНЧИКИ (П),2000563591,ПРИКАССА,G50,1,1,1.0
112301,ЦФО,TOWER,БАКАЛЕЯ,ПРИКАССА,ШОКОЛАД (П),ШОКОЛАДНЫЕ БАТОНЧИКИ (П),DI00078665,ПРИКАССА,G50,1,1,1.0
112302,ЦФО,TOWER,БАКАЛЕЯ,ПРИКАССА,ШОКОЛАД (П),ШОКОЛАДНЫЕ БАТОНЧИКИ (П),DI00079081,ПРИКАССА,G50,1,1,1.0


In [17]:
now_plng.sample()

,FO,SiteFormat,Уровень 2,Уровень 3,Уровень 4,Уровень 5,Item,Plng,Plng_Size,Count_TT,Count_TT_max,%TT
64934,СЗФО,DIXY,ФРЕШ,МОЛОЧНЫЕ ПРОДУКТЫ,СНЕКОВАЯ МОЛОЧНАЯ ПРОДУКЦИЯ,СНЕКИ МОЛОЧНЫЕ,2000000034,МОЛ.ГОР ЙОГУРТЫ,H2,31,31,1.0


In [18]:
now_plng_s = now_plng.groupby(['FO','SiteFormat', 'Plng', 'Plng_Size']).agg('%TT').sum() #находим сумму взвесов
now_plng_sum = pd.merge(now_plng, now_plng_s, on=['FO', 'Plng', 'Plng_Size','SiteFormat'], how='inner') #сумму взвесов припрепляем к общей табл.
#print(now_plng_sum.query('PLNG_SIZE=="M1" & Item=="2000591774"').head(3))
now_plng_sum.sample()

,FO,SiteFormat,Уровень 2,Уровень 3,Уровень 4,Уровень 5,Item,Plng,Plng_Size,Count_TT,Count_TT_max,%TT_x,%TT_y
150382,ЦФО,DIXY,ФРЕШ,МОЛОЧНЫЕ ПРОДУКТЫ,СНЕКОВАЯ МОЛОЧНАЯ ПРОДУКЦИЯ,СНЕКИ МОЛОЧНЫЕ,2000208731,МОЛ.ГОР ЙОГУРТЫ,H2,36,41,0.878049,177.707317


In [19]:
now_plng_s.sample()

FO    SiteFormat  Plng              Plng_Size
СЗФО  DIXY        ОРЕХИ СУХОФРУКТЫ  M1           32.0
Name: %TT, dtype: float64

In [20]:
now_plng_sum_u = now_plng_sum[['FO', 'Plng','SiteFormat', 'Plng_Size','%TT_y']].drop_duplicates(inplace = False) #удаляем дубликаты строк
now_plng_sum_u['Признак'] = '0!Сегодня' #создаем столбец в признаком Сегодня
now_plng_sum_u.sample()                  

,FO,Plng,SiteFormat,Plng_Size,%TT_y,Признак
34426,СЗФО,КОРОБКИ КОНФЕТ,DIXY,L4,19.0,0!Сегодня


In [21]:
now_plng_sum_u ['Size'] = now_plng_sum_u['Plng_Size']
now_plng_sum_u ['Size'] = now_plng_sum_u ['Size'].str.replace("T", "")
now_plng_sum_u ['Size'] = now_plng_sum_u ['Size'].str.replace("L", "")
now_plng_sum_u ['Size'] = now_plng_sum_u ['Size'].str.replace("M", "")
now_plng_sum_u ['Size'] = now_plng_sum_u ['Size'].str.replace("H", "")
now_plng_sum_u ['Size'] = now_plng_sum_u ['Size'].str.replace("G", "")
now_plng_sum_u ['Size'] = now_plng_sum_u ['Size'].str.replace("DS", "")
now_plng_sum_u.sample() 

,FO,Plng,SiteFormat,Plng_Size,%TT_y,Признак,Size
114462,ЦФО,ШОКОЛАД,GO,G20,19.0,0!Сегодня,20


In [22]:
display(now_plng_sum_u['Size'].unique())
display(now_plng_sum_u['SiteFormat'].unique())
now_plng_sum_u.info()

array(['20', '4', '21', '2', '1', '0', '3', '5', '6', '7', '22', '10',
       '12', '9', '8', '11', 'D77', '50'], dtype=object)

array(['GO', 'DIXY', 'DARKSTORE', 'TOWER'], dtype=object)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2094 entries, 0 to 153815
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   FO          2094 non-null   object 
 1   Plng        2094 non-null   object 
 2   SiteFormat  2094 non-null   object 
 3   Plng_Size   2094 non-null   object 
 4   %TT_y       2094 non-null   float64
 5   Признак     2094 non-null   object 
 6   Size        2094 non-null   object 
dtypes: float64(1), object(6)
memory usage: 130.9+ KB


In [23]:
display(quots['Size'].unique())
display(quots['SiteFormat'].unique())

array(['0', '1', '2', '3', '20', '21', '50', '77', '4', '5', '6', '7',
       '8', '9', '10', '11', '12', '22', '23'], dtype=object)

array(['DIXY'], dtype=object)

In [24]:
def determine_site_format(Size):
    if Size in ['0','1','2','3','4','5','6','7','8','9','10','11','12']:
        return 'DIXY'
    elif Size in ['20','21']:
        return 'GO'
    elif Size == '77':
        return 'DARKSTORE'
    elif Size == '50':
        return 'TOWER'
    else:
        return ''

In [25]:
quots['SiteFormat'] = quots['Size'].apply(determine_site_format)

In [26]:
quots['Size'] = quots['Size'].replace('77', 'D77')

In [27]:
display(quots['Size'].unique())
display(quots['SiteFormat'].unique())

array(['0', '1', '2', '3', '20', '21', '50', 'D77', '4', '5', '6', '7',
       '8', '9', '10', '11', '12', '22', '23'], dtype=object)

array(['DIXY', 'GO', 'TOWER', 'DARKSTORE', ''], dtype=object)

In [28]:
now_quots = pd.merge(now_plng_sum_u, quots, on=['Plng', 'Size','SiteFormat'], how='inner')
now_quots['Значение'] = now_quots['QTA_RG']-now_quots['%TT_y']
now_quots = now_quots [['FO', 'Plng', 'Plng_Size', 'Значение', 'Признак', 'SiteFormat']]
#now_quots.head() # таблица со значениями на сегодня
now_quots.sample()            

,FO,Plng,Plng_Size,Значение,Признак,SiteFormat
1543,ЦФО,МЯСО.ПТИЦА.ОХЛ,M9,39.666667,0!Сегодня,DIXY


In [29]:
#now_quots['Size'].unique()

In [30]:
vivod_plng.sample() 

,FO,SiteFormat,Уровень 2,Уровень 3,Уровень 4,Уровень 5,Item,Plng,Plng_Size,Признак,Count_TT,Count_TT_max,%TT
3023,ЦФО,DIXY,НОНФУД 2,СЕЗОННЫЕ ТОВАРЫ,САД,СЕМЕНА,2000608706,СПЕЦ ОБОР,H1,18,1,1,1.0


In [31]:
#матрица(вывод_plng)
vivod_plng = vivod_plng[['Item', 'FO', 'Plng', 'Plng_Size', '%TT','Признак', 'SiteFormat']]
vivod_plng = vivod_plng.rename(columns = {'%TT':'ЗначениеВывод'}) #vivod_plng  #7939 rows
vivod_quots = vivod_plng.groupby(['FO', 'Plng', 'Plng_Size', 'Признак', 'SiteFormat'])['ЗначениеВывод'].sum().reset_index() #vivod_quots #1865rows

In [32]:
vivod_quots.sample()  

,FO,Plng,Plng_Size,Признак,SiteFormat,ЗначениеВывод
434,СЗФО,ПИВО+САН,T1,15,DIXY,1.0


In [33]:
#матрица(ввод_plng)
vvod_plng = vvod_plng[['Item', 'FO', 'Plng', 'Plng_Size', '%TT','Признак', 'SiteFormat']]
vvod_plng = vvod_plng.rename(columns = {'%TT':'ЗначениеВвод'}) #vvod_plng  #2457 rows
vvod_quots = vvod_plng.groupby(['FO', 'Plng', 'Plng_Size', 'Признак', 'SiteFormat'])['ЗначениеВвод'].sum().reset_index() #vvod_quots #1865rows

vivod_plng1 = vivod_quots[['FO', 'Plng', 'Plng_Size','Признак', 'SiteFormat']] # 1865 rows
vvod_plng1 = vvod_quots[['FO', 'Plng', 'Plng_Size','Признак', 'SiteFormat']] # 974 rows

In [34]:
vvod_plng1.sample()  

,FO,Plng,Plng_Size,Признак,SiteFormat
589,ЦФО,ВОДА,L3,12,DIXY


In [35]:
# создаем структуру vod:
frames = [vivod_plng1, vvod_plng1]
vod = pd.concat(frames) # 10396 rows 
vods = vod.drop_duplicates(inplace = False) # 2386 rows
#vods.head(3) # структура 2386 строк#vvod_quots.head(3) #vivod_quots.head(3)

vvod_vivod = pd.merge(vods, vvod_quots, on=['FO', 'Plng', 'Plng_Size', 'Признак', 'SiteFormat'], how='left')
vvod_vivod_yes = pd.merge(vvod_vivod, vivod_quots, on=['FO', 'Plng', 'Plng_Size', 'Признак', 'SiteFormat'], how='left')

vvod_vivod_yes['ЗначениеВвод'] = vvod_vivod_yes['ЗначениеВвод'].fillna(0)
vvod_vivod_yes['ЗначениеВывод'] = vvod_vivod_yes['ЗначениеВывод'].fillna(0)
vvod_vivod_yes['Значение'] = vvod_vivod_yes['ЗначениеВвод'] - vvod_vivod_yes['ЗначениеВывод']
vv = vvod_vivod_yes[['FO', 'Plng', 'Plng_Size', 'Значение', 'Признак', 'SiteFormat']]

In [36]:
vvod_vivod_yes.sample()

,FO,Plng,Plng_Size,Признак,SiteFormat,ЗначениеВвод,ЗначениеВывод,Значение
2322,СЗФО,СПЕЦ ОБОР,H1,14,DIXY,5.333333,0.0,5.333333


In [37]:
#общие данные (объединяем таблицы)
data = pd.concat([now_quots, vv]) # 3784 rows  
#data

In [38]:
data['SiteFormat'].unique()

array(['GO', 'DIXY', 'DARKSTORE', 'TOWER'], dtype=object)

In [39]:
now_quots

,FO,Plng,Plng_Size,Значение,Признак,SiteFormat
0,СЗФО,ВИНО,G20,0.000000,0!Сегодня,GO
1,ЦФО,ВИНО,G20,0.000000,0!Сегодня,GO
2,СЗФО,ВИНО,M4,90.235294,0!Сегодня,DIXY
3,СЗФО,ВИНО,H4,90.043478,0!Сегодня,DIXY
4,ЦФО,ВИНО,M4,89.507463,0!Сегодня,DIXY
...,...,...,...,...,...,...
2008,ЦФО,РЫБНАЯ ГАСТРОНОМИЯ,G21,0.000000,0!Сегодня,GO
2009,ЦФО,ИКОРНИК,D77,12.000000,0!Сегодня,DARKSTORE
2010,ЦФО,ИКОРНИК,G21,4.000000,0!Сегодня,GO
2011,ЦФО,РЫБНАЯ ГАСТРОНОМИЯ,G50,-4.000000,0!Сегодня,TOWER


In [40]:
vv.query('Plng=="МАКАРОНЫ"')

,FO,Plng,Plng_Size,Значение,Признак,SiteFormat
239,СЗФО,МАКАРОНЫ,H1,-1.0,14,DIXY
240,СЗФО,МАКАРОНЫ,H2,-1.0,14,DIXY
241,СЗФО,МАКАРОНЫ,H3,-1.0,14,DIXY
242,СЗФО,МАКАРОНЫ,H3,-1.0,18,DIXY
243,СЗФО,МАКАРОНЫ,L1,-1.0,14,DIXY
244,СЗФО,МАКАРОНЫ,L2,-1.0,14,DIXY
245,СЗФО,МАКАРОНЫ,L3,-1.0,14,DIXY
246,СЗФО,МАКАРОНЫ,L3,-1.0,18,DIXY
247,СЗФО,МАКАРОНЫ,M1,-1.0,14,DIXY
248,СЗФО,МАКАРОНЫ,M2,-1.0,14,DIXY


In [42]:
file_path=r"C:\Users\VYuYaroshenko\Desktop\Контроль квот ПАЙТОН\данные.xlsx"

#запись в Excel-файл датафрейма now_quots: 
data.to_excel(
    file_path, 
    sheet_name='Сегодня', # название листа Сегодня
    index=False, # не будут указаны индексы
    startrow=0, # место записи с 0й строки
    startcol=0 # место записи с 0й колонки
)